In [ ]:
import numpy as np
import pandas as pd

import wandb
from config import Config

wandb.init(project="NeuroWood2022-name", entity="nuclear_foxes_team")

In [2]:
import source
import models

In [3]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, SGD, lr_scheduler
from torch.utils.data import DataLoader


torch.manual_seed(7575)
np.random.seed(7575)
torch.cuda.empty_cache()

## Create Dataset

In [4]:
output_dataframe = pd.read_csv('sample_submission.csv')
output_dataframe['id'] = output_dataframe['id'].astype(str)
output_dataframe.head()

,id,class
0,1,NaN
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN


In [5]:
train_dataset = source.WoodDataset()
test_dataset = source.WoodDataset(create_train=False)

train_dataloader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, pin_memory=True, shuffle=True)
test_dataloader = DataLoader(test_dataset, pin_memory=True, batch_size=Config.BATCH_SIZE)

## Model

In [ ]:
model = models.Effnetb3()

lr = 8e-4
gamma = 0.6
parameters = {
    'optimizer' : Adam(model.parameters(), lr=lr),
    'criterion' : CrossEntropyLoss(),
    'n_epochs' : 15,
    'device' : Config.DEVICE
}

parameters['scheduler'] = lr_scheduler.ExponentialLR(parameters['optimizer'], gamma=gamma)

wandb_dict = {
    "Learning_rate": lr,
    "Gamma": gamma,
    "Epochs": parameters['n_epochs'],
    "Batch_size": Config.BATCH_SIZE,
    "Network": "EFFnetB3 + Adam",
    "Full train dataset" : True
}

wandb.config.update(wandb_dict)

In [ ]:
source.train(model, train_dataloader, **parameters)
path = source.save_model(model, wandb_dict['Network'])
wandb_dict["Path"] = path
wandb.config.update(wandb_dict)

In [ ]:
predicts = source.predict(model, test_dataloader)

In [ ]:
output_dataframe['class'] = predicts
output_dataframe['class'] = output_dataframe['class'].astype(int)

In [ ]:
def transform(x):
    if (x == 0):
        return x
    elif (x == 1):
        return x
    else:
        return 3

In [ ]:
output_dataframe['class'] = output_dataframe['class'].apply(transform)
output_dataframe.to_csv('prediction.csv', index=False)